# The SVM vs. Logistic Regression Showdown

In this lab, you will practice working with non-linear kernels combined with logistic regression and SVM classifiers. The goal is to compare these commonly used techniques. Which comes out on top in terms of accuracy? Runtime? Is there much of a difference at all? Is the dominance of the SVM classifier in machine learning pedagogy justified? 

## Loading the Data

First, we load all the packages we'll need.

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics.pairwise import pairwise_kernels
import scipy
from sklearn import svm, linear_model
from sklearn.model_selection import GridSearchCV
import time

Again we download the data from the Tensorflow package, which you will need to install.  You can get the data from other sources as well. 

In the Tensorflow dataset, the training and test data are represented as arrays:

     Xtr.shape = 60000 x 28 x 28
     Xts.shape = 10000 x 28 x 28
     
The test data consists of `60000` images of size `28 x 28` pixels; the test data consists of `10000` images.

In [ ]:
import tensorflow as tf

(Xtr_raw,ytr),(Xts_raw,yts) = tf.keras.datasets.mnist.load_data()

print('Xtr shape: %s' % str(Xtr_raw.shape))
print('Xts shape: %s' % str(Xts_raw.shape))

ntr = Xtr_raw.shape[0]
nts = Xts_raw.shape[0]
nrow = Xtr_raw.shape[1]
ncol = Xtr_raw.shape[2]

Xtr shape: (60000, 28, 28)
Xts shape: (10000, 28, 28)


Each pixel value is from `[0,255]`.  For this lab, it will be convenient to recale the value to -1 to 1 and reshape it as a `ntr x npix` and `nts x npix`.

In [ ]:
npix = nrow*ncol
# print(npix)
Xtr = (Xtr_raw/255 - 0.5)
Xtr = Xtr.reshape((ntr,npix))
Xts = (Xts_raw/255 - 0.5)
Xts = Xts.reshape((nts,npix))

For this lab we're only going to use a fraction of the MNIST data -- otherwise our models will take too much time and memory to run. Using only part of the training data will of course lead to worse results. Given enough computational resources and time, we would ideally be running on the full data set. The follow code creates a new test and train set, with 10000 examples for train and 5000 for test. 

In [ ]:
ntr1 = 10000
nts1 = 5000
Iperm = np.random.permutation(ntr1)
Xtr1 = Xtr[Iperm[:ntr1],:]
ytr1 = ytr[Iperm[:ntr1]]
Iperm = np.random.permutation(nts1)
Xts1 = Xts[Iperm[:nts1],:]
yts1 = yts[Iperm[:nts1]]
# print(Xtr1)

## Problem set up and establishing a baseline

To simplify the problem (and speed things up) we're also going to restrict to binary classification. In particular, let's try to design classifier a that separates the 8's from all other digits.

Create binary 0/1 label vectors `ytr8` and `yts8` which are 1 wherever `ytr1` and `yts1` equal 8, and 0 everywhere else.

In [ ]:
# TODO

ytr8 = (ytr1 == 8).astype(int)
yts8 = (yts1 == 8).astype(int)


Most of the digits in the test dataset aren't equal to 8. So if we simply guess 0 for every image in `Xts`, we might expect to get classification accuracy around 90%. Our goal should be to significantly beat this **baseline**. 

Formally, write a few lines of code to check what test error would be achieved by the all zeros classifier.

In [ ]:
# TODO
# ...
# acc = 
# print('Accuaracy = {0:f}'.format(acc))

ypred8 = np.zeros(nts1)
acc = np.sum((yts8 == ypred8))/nts1
print('Accuaracy = {0:f}'.format(acc))


Accuaracy = 0.902200


As a second baseline, let's see how we do with standard (non-kernel) logistic regression. As in the MNIST demo, you can use `scikit-learn`'s built in function `linear_model.LogisticRegression` to fit the model and compute the accuracy. Use no regularization and the `lbfgs` solver. You should acheive an improvement to around 93-95%.

In [ ]:
# TODO
# ...
# acc = 
# print('Logistic Regression Accuaracy = {0:f}'.format(acc))


from sklearn import linear_model
logreg = linear_model.LogisticRegression(verbose=10, solver='lbfgs', multi_class='multinomial',max_iter=500)
logreg.fit(Xtr1,ytr8)

nts1 = 5000
Iperm_ts = np.random.permutation(nts) 
# Xts8 = Xts[Iperm_ts[:nts8],:]
# yts8 = yts[Iperm_ts[:nts8]]
yhat = logreg.predict(Xts1)
acc = np.mean(yhat == yts8)
print('Logistic Regression Accuaracy = {0:f}'.format(acc))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Logistic Regression Accuaracy = 0.942200


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   37.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   37.1s finished


## Kernel Logistic Regression

To improve on this baseline performance, let's try using the logistic regression classifier with a *non-linear* kernel. Recall from class that any non-linear kernel similarity function $k(\vec{w},\vec{z})$ is equal to $\phi(\vec{w})^T\phi(\vec{z})$ for some feature transformation $\phi$. However, we typically do not need to compute this feature tranformation explicitly: instead we can work directly with the kernel gram matrix $K \in \mathbb{R}^{n\times n}$. Recall that $K_{i,j} = k(\vec{x}_i,\vec{x}_j)$ where $\vec{x}_i$ is the $i^\text{th}$ training data point. 

For this lab we will be using the radial basis function kernel. For a given scaling factor $\gamma$ this kernel is defined as:
$$
k(\vec{w},\vec{z}) = e^{-\gamma\|\vec{w}-\vec{z}\|_2^2}
$$

In [ ]:
def rbf_kernel(w,z,gamma):
    d = w - z
    return np.exp(-gamma*np.sum(d*d))

Construct the kernel matrix `K1` for `Xtr1` with `gamma = .05`.

In [ ]:
import collections
# TODO 
gamma = .05
K1 = np.zeros((Xtr1.shape[0], Xtr1.shape[0]), dtype=float)
for row in range(Xtr1.shape[0]):
  for col in range(row, Xtr1.shape[0]):
    K1[row][col] = rbf_kernel(Xtr1[row], Xtr1[col], gamma)
    K1[col][row] = K1[row][col]

If you used a for loop (which is fine) your code might take several minutes to run! Part of the issue is that Python won't know to properly parallize your for loop. For this reason, when constructing kernel matrices it is often faster to us a built-in, carefully optimized function with explicit parallelization. Scikit learn provides such a function through their `metrics` library. 

Referring to the documentation here
https://scikit-learn.org/stable/modules/metrics.html#metrics, use this built in function to recreate the same kernel matrix you did above. Store the result at `K`.

In [59]:
# TODO
# K = 
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import rbf_kernel as rbf 
K = rbf(Xtr1, Xtr1, gamma = 0.05)



Check that you used the function correctly by writing code to confirm that `K = K1`, or at least that the two are equal up to very small differences (which could arise due to numerical precision issues). Try to do this **without a for loop** for full credit. You will get partial credit if you use a for loop.

In [60]:
# TODO

# Just testing. No need to rerun
# np.allclose(K1, K)

True

When using a non-linear kernel, it is important to check that you have chosen reasonable parameters (in our case the only parameter is `gamma`). We typically do not want $k(\vec{x}_i,\vec{x}_j)$ to be either negligably small, or very large for all $i\neq j$ in our data set, or we won't be able to learn anything. For the RBF kernel this means that, for any $\vec{x}_i$ we don't want $k(\vec{x}_i,\vec{x}_j)$ very close to 1 (e.g. .9999) for all $j$, or very close to $0$ (e.g. 1e-8) for all $j$. 

Let's just check that we're in good shape for the first data vector $\vec{x}_0$. Do so by printing out the 10 largest and 10 smallest values of $k(\vec{x}_0,\vec{x}_j)$ for $j\neq 0$. Note that we always have $k(\vec{x}_0,\vec{x}_0) = 1$ for the RBF kernel. 

In [61]:
# TODO
K0 = np.sort(K1[0])
print('Maximum similarities: \n' + str(K0[-10:]))
print('Minimum similarities: \n' + str(K0[:10]))

Maximum similarities: 
[0.32684629 0.32921906 0.33025251 0.33890633 0.36547863 0.38108794
 0.43343582 0.48617994 0.49642839 1.        ]
Minimum similarities: 
[3.76381749e-05 4.14103694e-05 6.18902665e-05 6.87354366e-05
 7.88196421e-05 7.88287943e-05 8.82068925e-05 9.44521551e-05
 9.48388146e-05 1.11091744e-04]


In [62]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [63]:
import pickle

with open( "/content/drive/MyDrive/mnist_logreg.p", "wb" ) as fp:
    pickle.dump([K1, K], fp)

In [64]:
import pickle

with open( "/content/drive/MyDrive/mnist_logreg.p", "rb" ) as fp:
    K1, K = pickle.load(fp)

### Implementation
Maybe surprisingly Scikit learn does not have an implementation for kernel logistic regression. So we have to implement our own!

Write a function function `log_fit` that minimizes the $\ell_2$-regularized logisitic regression loss:
$$
L(\boldsymbol{\alpha}) =\sum_{i=1}^n (1-y_i)(\phi(\mathbf{x}_i)^T\phi(\mathbf{X})^T\vec{\alpha}) - \log(h(\phi(\mathbf{x}_i)^T\phi(\mathbf{X})^T\boldsymbol{\alpha}) + \lambda \|\phi(\mathbf{X})^T\boldsymbol{\alpha}\|_2^2.
$$
As input it takes an $n\times n$ kernel matrix $K$ for the training data, an $n$ length vector `y` of binary class labels, and regularization parameter `lamb`.

To implement your function you can either use your own implmentation of gradient descent or used a built in minimizer from `scipy.optimize.minimize`. I recommend the later approach: as we saw in the last lab, gradient descent can converge slowly for this objective.

In [65]:
import scipy 

def Leval_reg_linear_kernel(alpha, K, y, lamb):
  z = K@alpha
  h = 1/(1+np.exp(-z))
  Lr = np.sum((1-y)*z - np.log(h)) + lamb*(alpha.T)@z
  # Lrgrad = K@(h-y) + 2*lamb*z
  Lrgrad = K@(h-y) + 2*lamb*z
  return Lr, Lrgrad 

In [66]:
# # TODO
def log_fit(K,y,lamb,nit=1000):
  f = lambda alpha: Leval_reg_linear_kernel(alpha, K, y, lamb)[0]
  g = lambda alpha: Leval_reg_linear_kernel(alpha, K, y, lamb)[1]
  alpha0 = np.zeros(ntr1)
  res = scipy.optimize.minimize(f, alpha0, args = (), method = 'tnc', jac = g, options = {'maxIter': nit})
  return res.x
  

Use the `log_fit` function defined above to find parameters `alpha` for the kernel logistic regression model using `lamb = 0` and `K` as constructed above (with `gamma = .05`). 

In [69]:
# TODO
alpha = log_fit(K,ytr8,lamb = 0,nit=1000)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: OptimizeWarning: Unknown solver options: maxIter
  


In [70]:
alpha.shape

(10000,)

Suppose we have a test dataset with $m$ examples $\vec{w}_1,\ldots, \vec{w}_m$. Once we obtain a coefficient vector $\alpha$, making predictions for any $\vec{w}_j$ in the test set requires computing:
$$
{y}_{j} = \sum_{i=1}^n \alpha_i \cdot k(\vec{w}_{j}, \vec{x}_i).
$$
where $\vec{x}_1, \ldots \vec{x}_n$ are our training data vectors. We classify $\vec{w}_{j}$ in class 0 if ${y}_{j} \leq 0$ and in class 1 if ${y}_{j} > 0$. 

This computation can be rewritten in matrix form as follows:
$$
\vec{y}_{test} = K_{test}\vec{\alpha}, 
$$
where $\vec{y}_{text}$ is an $m$ length vector and $K_{test}$ is a $m\times n$ matrix whose $(j,i)$ entry is equal to $k(\vec{w}_{j}, \vec{x}_i)$. We classify $\vec{w}_{j}$ in class 0 if $\vec{y}_{test}[j] \leq 0$ and in class 1 if $\vec{y}_{test}[j] > 0$. 


Use the `pairwise_kernels` function to construct $K_{test}$. Then make predictions for the test set and evaluate the accuracy of our kernel logistic regression classifier. You should see a pretty substantial lift in accuracy to around $97\%$

In [71]:
from sklearn.metrics import pairwise
# TODO 
# t = time.time()
Ktest = pairwise_kernels(Xts1, Xtr1, metric = "rbf", gamma = 0.05)
# yhat = Ktest@alpha
# return
# Ktest = ...

In [72]:
# TODO
# yhat = ... (vector containing predicted 0,1 labels)
# acc = np.mean(yhat == yts8)
# print("Test accuracy = %f" % acc)

yhat = Ktest@alpha
print(yhat)
print(yts8)
yhat = np.where(yhat <= 0,0,1)
# np.where(x < y, x, 10 + y)
acc = np.mean(yhat == yts8)
print("Test accuracy = %f" % acc)



[-34.2416466  -20.15838618 -25.80371192 ... -25.99536714 -49.63691286
 -26.37014087]
[0 0 0 ... 0 0 0]
Test accuracy = 0.981000


## Kernel Support Vector Machine

The goal of this lab is to compare Kernel Logistic Regression to Kernel Support Vector machines. Following `demo_mnist_svm.ipynb` create and train an SVM classifier on `Xtr1` and `ytr8` using an RBF kernel with `gamma = .05` (the same value we used for logistic regression aboe). Use margin parameter `C = 10`.

In [75]:
# TODO

# gamma = [0.005]
t = time.time()
svcrbf = svm.SVC(probability= False, kernel = "rbf", C = 10, gamma = 0.05, verbose = 10)
svcrbf.fit(Xtr1, ytr8)
yhat_ts = svcrbf.predict(Xts1)
ysvm = yhat_ts
acc = np.mean(ysvm == yts8)
elapsed = time.time() - 1


[LibSVM]

Calculate and print the accuracy of the SVM classifier. You should obtain a similar result as for logistic regression: something close to $97\%$ accuracy. 

In [77]:
# TODO
# ysvm = ... (vector containing predicted 0,1 labels)
yhat_ts = svcrbf.predict(Xts1)
ysvm = yhat_ts
acc = np.mean(yhat == yts8)
print("Test accuracy = %f" % acc)  

Test accuracy = 0.977600


## The Showdown 

Both SVM classifiers and kernel logisitic regression require tuning parameters to obtain the best possible result. In our setting we will stick with an RBF kernel (although this could be tuned). So we only consider tuning the kernel width parameter `gamma`, as well as the regularization parameter `lamb` for logistic regression, and the margin parameter `C` for SVM. We will choose parameters using for-loops and train-test cross validation.

Train a logistic regression classifier with **all combinations** of the parameters included below in vectors `gamma` and `lamb`. For each setting of parameters, compute and print:
* the test error obtained
* the total runtime of classification in seconds (including training time and prediction time)

For computing runtime you might want to use the `time()` function from the `time` library, which we already imported ealier. 

In [ ]:
gamma = [.1, .05,.02,.01,.005]
lamb = [0,1e-6,1e-4,1e-2]
# TODO
# ...
for i in range(len(gamma)): 
  t = time.time()
  K = pairwise_kernels(Xtr1, Xtr1, metric = "rbf", gamma = gamma[i])
  for j in range(len(lamb)):
    alpha = log_fit(K, ytr8, lamb[j], nit = 1000)
    Ktest = pairwise_kernels(Xts1, Xtr1, metric = "rbf", gamma = gamma[i])
    # yStorage = np.zeros(yts8.shape[0])
    yhat = np.zeros(yts8.shape[0])
    for k in range(yts8.shape[0]):
      curr_yhat = alpha@Ktest[k, :]
      if curr_yhat >0: 
        yhat[k] = 1 
    acc = np.mean(yhat == yts8)
    elapsed = time.time() - t 
    print('Gamma is ' + str(gamma[i]))
    print('Lamb is ' + str(lamb[j]))
    print('The acc is ' + str(acc))
    print('The runtime is ' + str(elapsed))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: OptimizeWarning: Unknown solver options: maxIter
  


Gamma is 0.1
Lamb is 0
The acc is 0.9634
The runtime is 50.194396018981934


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: OptimizeWarning: Unknown solver options: maxIter
  


Gamma is 0.1
Lamb is 1e-06
The acc is 0.9658
The runtime is 154.05476665496826


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: OptimizeWarning: Unknown solver options: maxIter
  


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: OptimizeWarning: Unknown solver options: maxIter
  


Gamma is 0.02
Lamb is 1e-06
The acc is 0.985
The runtime is 1879.7416179180145


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: OptimizeWarning: Unknown solver options: maxIter
  


Gamma is 0.02
Lamb is 0.0001
The acc is 0.9844
The runtime is 3808.1090004444122


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: OptimizeWarning: Unknown solver options: maxIter
  


TODO: What was the best test error achieved, and what setting of parameters achieved this error? Was the kernel logistic regression classifier more sensitive to changes in `gamma` or `lamb`? Discuss in 1-3 short sentences below.

Best performance error achieved: 0.98. 

It is more sensitive to gamma, because the acc rises as gamma becomes smaller. Whereas for changes in lamb, it is usually changes too, but not as significantly as what gamma does to acc. 



Now let's do the same thing for the kernel Support Vector Classifier. Train an SVM classifier with **all combinations** of the parameters included below in vectors `gamma` and `C`. For each setting of parameters, compute:
* the test error obtained
* the total runtime of classification in seconds (including training time and prediction time)

In [88]:
gamma = [.1, .05,.02,.01,.005]
C = [.01,.1,1,10]

for i in range(len(gamma)):
  t = time.time()
  for j in range(len(C)):
    svcrbf = svm.SVC(probability=False, kernel = "rbf", C = C[j], gamma = gamma[i], verbose = 10)
    svcrbf.fit(Xtr1, ytr8)
    yhat_ts = svcrbf.predict(Xts1)
    ysvm = yhat_ts
    acc = np.mean(ysvm == yts8)
    elapsed = time.time() - t
    print('Gamma is ' + str(gamma[i]))
    print('Lamb is ' + str(lamb[j]))
    print('The acc is ' + str(acc))
    print('The runtime is ' + str(elapsed))
# TODO
# ...

[LibSVM]Gamma is 0.1
Lamb is 0
The acc is 0.9022
The runtime is 74.9006564617157
[LibSVM]Gamma is 0.1
Lamb is 1e-06
The acc is 0.9022
The runtime is 184.75584721565247
[LibSVM]Gamma is 0.1
Lamb is 0.0001
The acc is 0.9282
The runtime is 321.3443694114685
[LibSVM]Gamma is 0.1
Lamb is 0.01
The acc is 0.9342
The runtime is 476.0332658290863
[LibSVM]Gamma is 0.05
Lamb is 0
The acc is 0.9022
The runtime is 26.92007875442505
[LibSVM]Gamma is 0.05
Lamb is 1e-06
The acc is 0.9228
The runtime is 53.38871169090271
[LibSVM]Gamma is 0.05
Lamb is 0.0001
The acc is 0.9774
The runtime is 84.71781659126282
[LibSVM]Gamma is 0.05
Lamb is 0.01
The acc is 0.9802
The runtime is 118.57894682884216
[LibSVM]Gamma is 0.02
Lamb is 0
The acc is 0.9022
The runtime is 16.961200952529907
[LibSVM]Gamma is 0.02
Lamb is 1e-06
The acc is 0.9512
The runtime is 31.888372659683228
[LibSVM]Gamma is 0.02
Lamb is 0.0001
The acc is 0.982
The runtime is 43.28492593765259
[LibSVM]Gamma is 0.02
Lamb is 0.01
The acc is 0.9858
The

TODO: What was the best test error achieved, and what setting of parameters achieved this error? Which performed better in terms of accuracy, the SVM or logisitic regression classifier? How about in terms of runtime?

The best test error achieved: 0.987

The setting of parameters: 
[LibSVM]Gamma is 0.01
Lamb is 0.01
The acc is 0.987
The runtime is 50.9332857131958

The logistic regression performed better, but in terms of runtime, the SVM performed better. 


**NOTE:** For `sklearns`'s built in classifiers, including svm.SVC, there is a function called `GridSearchCV` which can automatically perform hyperparamater tuning for you. The main advantage of the method (as opposed to writing for-loops) is that it supports parallelization, so it can fit with different parameters at the same time. The function also supports automatic $k$-fold cross-validation (instead of simple train/test split). 

You might be interested in using this function in the future. If so, please check out the tutorial in the following lab from previous year: https://github.com/sdrangan/introml/blob/master/unit08_svm/lab_emnist_partial.ipynb.